In [1]:
import pandas as pd
import random
import json
import xlrd
import random
from math import cos, asin, sqrt, pi
import pandas as pd  

In [2]:
internationals = [5, 12, 15, 22, 37, 88]

In [3]:
years_in_future = 2
age_tid = [(-2, 18), (-4, 17), (-5, 16)]


In [4]:
age = age_tid[years_in_future][1]
diff = age-18
tid = age_tid[years_in_future][0]

In [5]:
def getJSON(filePath):    
    with open(filePath, encoding='utf-8-sig') as fp:
        return json.load(fp)


In [6]:
def writeJSON(filePath, obj):
    with open(filePath, 'r+',encoding='utf-8') as fp:
        json.dump(obj, fp, indent=4)

In [7]:
fileName = 'NCBCA 2062 NT Bracket WITH RECRUITS.json'
obj = getJSON(fileName)
year = int(obj.get("meta")["phaseText"][0:4]) 


In [8]:
cols = ['Name','Hgt', 'Ovr']
class_df = pd.DataFrame(columns = cols)

In [9]:
i = 0
for player in obj.get("players"):
    ratings = player["ratings"][-1];   
    if ratings["season"] == year - diff and player["tid"] == tid and year - player.get("born")["year"] == age:
        name = player["firstName"] + " " + player["lastName"]
        class_df.loc[i, "Name"] = name
        class_df.loc[i, "Ovr"] = ratings['ovr']
        class_df.loc[i, "Hgt"] = ratings["hgt"]
        i+=1
class_df = class_df.sort_values(by=['Ovr'], ascending = False)
class_df = class_df.reset_index(drop=True)

In [10]:
def getType(hgt):
    randType = random.random()

    if (hgt >= 59):

        if (randType < 0.02):
            type = "captain"
        elif (randType < 0.09):
            type = "triple dub"
        elif (randType < 0.20):
            type = "fundamentals"
        elif (randType < 0.35):
            type = "athletic big"
        elif (randType < 0.55):
            type = "offensive big"
        elif (randType < 0.75):
            type = "stretch big"
        elif (randType < 0.95):
            type = "defensive big"
        else:
            type = "unicorn"
    elif (hgt <= 36):
        #6'3" or shorter
        if (randType < 0.03):
            type = "captain"
        elif (randType < 0.08):
            type = "triple dub"
        elif (randType < 0.20):
            type = "athletic guard"
        elif (randType < 0.32):
            type = "3_and_d"
        elif (randType < 0.42):
            type = "defense only"
        elif (randType < 0.50):
            type = "jhart"
        elif (randType < 0.60):
            type = "fundamentals"
        elif (randType < 0.65):
            type = "rondo"
        elif (randType < 0.78):
            type = "shooter"
        elif (randType < 0.90):
            type = "slasher"
        else:
            type = "pure point"
    else:
        if (randType < 0.03):
            type = "captain"
        elif (randType < 0.12):
            type = "3_and_d"
        elif (randType < 0.23):
            type = "athletic big"
        elif (randType < 0.33):
            type = "athletic guard"
        elif (randType < 0.43):
            type = "defense only"
        elif (randType < 0.53):
            type = "fundamentals"
        elif (randType < 0.63):
            type = "shooter"
        elif (randType < 0.72):
            type = "slasher"
        elif (randType < 0.80):
            type = "triple dub"
        elif (randType < 0.90):
            type = "derozan"
        else:
            type = "versatile"
    return type



In [11]:
def getBaselineAdj(row):
    ratingsAdjustment = random.randint(-15, 15)
    return ratingsAdjustment


def bound(low, high, value):
    return max(low, min(high, value))


def getFactor(row):
    factor = bound(0.2, 1.2, random.gauss(1., 0.2))
    return factor

def getFuzz(value):
    factor = random.gauss(value, 3)
    return factor


def getPretty(value):
    final = round(bound(0.,100.,value))
    return final

def getComposite(ratings, weights):
    num = 0
    denom = 0
    for r, w in zip(ratings, weights):
        num += w * r
        denom += 100 * w
    return num/denom

def calcPos(ratings):
    position = ""
    if ratings["hgt"] >= 59:
        position = "C"
    elif ratings["hgt"] >= 52:
        position = "PF"
    elif ratings["hgt"] >= 44:
        position = "SF"
    elif ratings["hgt"] >= 30:
        position = "SG"
    else:
        position = "G"
        
    pg = sg = sf = pf = c = False
    if (ratings["spd"] >= 60 and ratings["pss"] >= 50 and ratings["drb"] >= 50) or (ratings["spd"] >= 40 and ratings["pss"] >= 65 and ratings["drb"] >= 65):
        pg = True
    if ratings["spd"] >= 50 and ratings["drb"] >= 50 and ratings["hgt"] >= 30 and (ratings["dnk"] >= 58 or ratings["tp"] > 63):
        sg = True
    if ratings["spd"] >= 40 and ratings["drb"] >= 30 and ratings["hgt"] >= 44 and (ratings["dnk"] >= 58 or ratings["tp"] > 63):
        sg = True
    if ratings["hgt"] >= 44 and ratings["stre"] >= 55 and (ratings["hgt"] + ratings["stre"] >= 110) and (ratings["hgt"] <= 63 or ratings["tp"] > 60):
        pf = True
    if ratings["hgt"] >= 63 or (ratings["hgt"] >= 54 and ratings["stre"] >= 75):
        c = True
        pg = False
        sg = False
    
    if pg and not sg and not sf and not pf and not c:
        position = "PG"
    elif not pg and sg and not sf and not pf and not c:
        position = "SG"
    elif not pg and not sg and sf and not pf and not c:
        position = "SF"
    elif not pg and not sg and not sf and  pf and not c:
        position = "PF"
    elif not pg and not sg and not sf and not pf and  c:
        position = "C"
    
    if c and (pg or sg):
        position = "F"
    elif (pg or sg) and (sf or pf):
        position = "GF"
    elif c and (pf or sf):
        position = "FC"
    elif pf and sf:
        position = "F"
    elif pg and sg:
        position = "G"

    return position
        
    
def getAtts(type):
    cutoff = 0.61
    if type == '3':
        ratings = ['Oiq', 'Tp']
        weights = [0.1, 1]
        cutoff = 0.59
    if type == 'A':
        ratings = ["Stre", "Spd", "Jmp", "Hgt"]
        weights = [1, 1, 1, 0.75]
        cutoff = 0.63
    if type == 'B':
        ratings = ['Drb', 'Spd']
        weights = [1, 1]
        cutoff = 0.68
    if type == 'Di':
        ratings = ["Hgt", "Stre", "Spd", "Jmp", "Diq"]
        weights = [2.5, 1, 0.5, 0.5, 2]
        cutoff = 0.57
    if type == 'Dp':
        ratings = ["Hgt", "Stre", "Spd", "Jmp", "Diq"]
        weights = [0.5, 0.5, 2, 0.5, 1]
    if type == 'Po':
        ratings = ["Hgt", "Stre", "Spd", "Ins", "Oiq"]
        weights = [2, 0.6, 0.2, 1, 0.2]
    if type == 'Ps':
        ratings = ["Drb", "Pss", "Oiq"]
        weights = [0.1, 1]
        cutoff = 0.63
    if type == 'R':
        ratings = ["Hgt", "Stre", "Jmp", "Reb", "Oiq", "Diq"]
        weights = [2, 0.1, 0.1, 2, 0.5, 0.5]
    return (ratings, weights, cutoff)

def getTypeFactor(rating, playerType, ratingName):
    value_returned = rating
    if playerType == 'pure point':
        if ratingName == 'Drb':
            value_returned = 1.4 * rating
        elif ratingName == 'Jmp':
            value_returned = 1.2 * rating
        elif ratingName == 'Spd':
            value_returned = 1.4 * rating
        elif ratingName == 'Ins':
            value_returned = 1.5 * rating
        elif ratingName == 'Pss':
            value_returned = 1.6 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.5 * rating
        elif ratingName == 'Reb':
            value_returned = 0.8 * rating
        elif ratingName == 'Diq':
            value_returned = 1.2 * rating
        elif ratingName == 'Tp':
            value_returned = 1.2 * rating
    elif playerType == 'rondo':
        if ratingName == 'Drb':
            value_returned = 1.4 * rating
        elif ratingName == 'Stre':
            value_returned = 0.7 * rating
        elif ratingName == 'Reb':
            value_returned = 0.8 * rating
        elif ratingName == 'Jmp':
            value_returned = 1.2 * rating
        elif ratingName == 'Spd':
            value_returned = 1.45 * rating
        elif ratingName == 'Ins':
            value_returned = 1.4 * rating
        elif ratingName == 'Pss':
            value_returned = 1.7 * rating
        elif ratingName == 'Ft':
            value_returned = 0.9 * rating
        elif ratingName == 'Fg':
            value_returned = 0.7 * rating
        elif ratingName == 'Tp':
            value_returned = 0.7 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.8 * rating
        elif ratingName == 'Diq':
            value_returned = 1.2 * rating
    elif playerType == 'athletic guard':
        if ratingName == 'Jmp':
            value_returned = 1.6 * rating
        elif ratingName == 'Spd':
            value_returned = 1.6 * rating
        elif ratingName == 'Stre':
            value_returned = 1.3 * rating
        elif ratingName == 'Dnk':
            value_returned = 1.5 * rating
        elif ratingName == 'Ins':
            value_returned = 1.4 * rating
        elif ratingName == 'Oiq':
            value_returned = 0.8 * rating
        elif ratingName == 'Endu':
            value_returned = 1.45 * rating
        elif ratingName == 'Pss':
            value_returned = 0.9 * rating
        elif ratingName == 'Drb':
            value_returned = 1.2 * rating
    elif playerType == 'triple dub':
        if ratingName == 'Jmp':
            value_returned = 1.3 * rating
        elif ratingName == 'Stre':
            value_returned = 1.2 * rating
        elif ratingName == 'Spd':
            value_returned = 1.15 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.2 * rating
        elif ratingName == 'Ins':
            value_returned = 1.25 * rating
        elif ratingName == 'Pss':
            value_returned = 1.35 * rating
        elif ratingName == 'Reb':
            value_returned = 1.3 * rating
        elif ratingName == 'Tp':
            value_returned = 1.2 * rating
    elif playerType == '3_and_d':
        if ratingName == 'Jmp':
            value_returned = 1.3 * rating
        elif ratingName == 'Spd':
            value_returned = 1.3 * rating
        elif ratingName == 'Stre':
            value_returned = 1.2 * rating
        elif ratingName == 'Drb':
            value_returned = 0.8 * rating
        elif ratingName == 'Pss':
            value_returned = 0.7 * rating
        elif ratingName == 'Ft':
            value_returned = 1.3 * rating
        elif ratingName == 'Fg':
            value_returned = 1.3 * rating
        elif ratingName == 'Tp':
            value_returned = 1.4 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.25 * rating
        elif ratingName == 'Diq':
            value_returned = 1.1 * rating
        elif ratingName == 'Endu':
            value_returned = 1.15 * rating
    elif playerType == 'captain':
        if ratingName == 'Jmp':
            value_returned = 1.3 * rating
        elif ratingName == 'Drb':
            value_returned = 1.2 * rating
        elif ratingName == 'Pss':
            value_returned = 1.2 * rating
        elif ratingName == 'Stre':
            value_returned = 1.2 * rating
        elif ratingName == 'Spd':
            value_returned = 1.35 * rating
        elif ratingName == 'Ins':
            value_returned = 1.3 * rating
        elif ratingName == 'Reb':
            value_returned = 1.2 * rating
        elif ratingName == 'Dnk':
            value_returned = 1.2 * rating
        elif ratingName == 'Ft':
            value_returned = 1.2 * rating
        elif ratingName == 'Fg':
            value_returned = 1.2 * rating
        elif ratingName == 'Tp':
            value_returned = 1.3 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.6 * rating
        elif ratingName == 'Diq':
            value_returned = 1.6 * rating
        elif ratingName == 'Endu':
            value_returned = 1.5 * rating
    elif playerType == 'fundamentals':
        if ratingName == 'Jmp':
            value_returned = 0.75 * rating
        elif ratingName == 'Spd':
            value_returned = 0.75 * rating
        elif ratingName == 'Drb':
            value_returned = 1.3 * rating
        elif ratingName == 'Pss':
            value_returned = 1.35 * rating
        elif ratingName == 'Ft':
            value_returned = 1.5 * rating
        elif ratingName == 'Ins':
            value_returned = 1.4 * rating
        elif ratingName == 'Fg':
            value_returned = 1.2 * rating
        elif ratingName == 'Tp':
            value_returned = 1.1 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.2 * rating
        elif ratingName == 'Diq':
            value_returned = 1.2 * rating
        elif ratingName == 'Reb':
            value_returned = 1.2 * rating
        elif ratingName == 'Endu':
            value_returned = 0.7 * rating
    elif playerType == 'defensive big':
        if ratingName == 'Stre':
            value_returned = 1.25 * rating
        elif ratingName == 'Reb':
            value_returned = 1.6 * rating
        elif ratingName == 'Ft':
            value_returned = 0.8 * rating
        elif ratingName == 'Dnk':
            value_returned = 1.35 * rating
        elif ratingName == 'Ins':
            value_returned = 1.3 * rating
        elif ratingName == 'Tp':
            value_returned = 0.6 * rating
        elif ratingName == 'Drb':
            value_returned = 0.6 * rating
        elif ratingName == 'Pss':
            value_returned = 0.7 * rating
        elif ratingName == 'Diq':
            value_returned = 1.55 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.3 * rating
    elif playerType == 'offensive big':
        if ratingName == 'Stre':
            value_returned = 1.4 * rating
        elif ratingName == 'Reb':
            value_returned = 1.4 * rating
        elif ratingName == 'Spd':
            value_returned = 0.9 * rating
        elif ratingName == 'Ins':
            value_returned = 1.5 * rating
        elif ratingName == 'Dnk':
            value_returned = 1.5 * rating
        elif ratingName == 'Tp':
            value_returned = 0.8 * rating
        elif ratingName == 'Fg':
            value_returned = 1.1 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.2 * rating
        elif ratingName == 'Diq':
            value_returned = 0.9 * rating
    elif playerType == 'athletic big':
        if ratingName == 'Stre':
            value_returned = 1.35 * rating
        elif ratingName == 'Jmp':
            value_returned = 1.55 * rating
        elif ratingName == 'Ins':
            value_returned = 1.4 * rating
        elif ratingName == 'Dnk':
            value_returned = 1.4 * rating
        elif ratingName == 'Reb':
            value_returned = 1.4 * rating
        elif ratingName == 'Ft':
            value_returned = 0.7 * rating
        elif ratingName == 'Fg':
            value_returned = 0.9 * rating
        elif ratingName == 'Tp':
            value_returned = 0.75 * rating
        elif ratingName == 'Oiq':
            value_returned = 0.75 * rating
        elif ratingName == 'Diq':
            value_returned = 0.75 * rating
        elif ratingName == 'Endu':
            value_returned = 1.4 * rating
    elif playerType == 'stretch big':
        if ratingName == 'Stre':
            value_returned = 1.1 * rating
        elif ratingName == 'Reb':
            value_returned = 0.7 * rating
        elif ratingName == 'Drb':
            value_returned = 0.8 * rating
        elif ratingName == 'Pss':
            value_returned = 0.8 * rating
        elif ratingName == 'Ft':
            value_returned = 1.25 * rating
        elif ratingName == 'Fg':
            value_returned = 1.3 * rating
        elif ratingName == 'Tp':
            value_returned = 1.4 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.3 * rating
        elif ratingName == 'Diq':
            value_returned = 0.9 * rating
    elif playerType == 'unicorn':
        if ratingName == 'Dnk':
            value_returned = 0.7 * rating
        if ratingName == 'Endu':
            value_returned = 0.7 * rating
        elif ratingName == 'Stre':
            value_returned = 0.6 * rating
        elif ratingName == 'Reb':
            value_returned = 0.8 * rating
        elif ratingName == 'Pss':
            value_returned = 1.4 * rating
        elif ratingName == 'Ft':
            value_returned = 1.5 * rating
        elif ratingName == 'Fg':
            value_returned = 1.3 * rating
        elif ratingName == 'Tp':
            value_returned = 1.35 * rating
        elif ratingName == 'Spd':
            value_returned = 0.65 * rating
        elif ratingName == 'Diq':
            value_returned = 0.9 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.2 * rating
    elif playerType == 'defense only':
        if ratingName == 'Stre':
            value_returned = 1.3 * rating
        elif ratingName == 'Jmp':
            value_returned = 1.5 * rating
        elif ratingName == 'Spd':
            value_returned = 1.4 * rating
        elif ratingName == 'Reb':
            value_returned = 1.4 * rating
        elif ratingName == 'Ft':
            value_returned = 0.6 * rating
        elif ratingName == 'Ins':
            value_returned = 1.2 * rating
        elif ratingName == 'Fg':
            value_returned = 0.6 * rating
        elif ratingName == 'Tp':
            value_returned = 0.6 * rating
        elif ratingName == 'Drb':
            value_returned = 0.8 * rating
        elif ratingName == 'Diq':
            value_returned = 1.65 * rating
        elif ratingName == 'End':
            value_returned = 1.3 * rating
    elif playerType == 'jhart':
        if ratingName == 'Stre':
            value_returned = 1.25 * rating
        elif ratingName == 'Reb':
            value_returned = 1.2 * rating
        elif ratingName == 'Endu':
            value_returned = 1.3 * rating
        elif ratingName == 'Ft':
            value_returned = 0.8 * rating
        elif ratingName == 'Fg':
            value_returned = 0.8 * rating
        elif ratingName == 'Dnk':
            value_returned = 1.2 * rating
        elif ratingName == 'Ins':
            value_returned = 1.1 * rating
        elif ratingName == 'Tp':
            value_returned = 1.1 * rating
        elif ratingName == 'Drb':
            value_returned = 0.8 * rating
        elif ratingName == 'Pss':
            value_returned = 1.1 * rating
        elif ratingName == 'Diq':
            value_returned = 1.3 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.3 * rating
    elif playerType == 'shooter':
        if ratingName == 'Stre':
            value_returned = 0.8 * rating
        elif ratingName == 'Jmp':
            value_returned = 1.1 * rating
        elif ratingName == 'Spd':
            value_returned = 1.1 * rating
        elif ratingName == 'Reb':
            value_returned = 0.8 * rating
        elif ratingName == 'Ft':
            value_returned = 1.6 * rating
        elif ratingName == 'Fg':
            value_returned = 1.55 * rating
        elif ratingName == 'Tp':
            value_returned = 1.5 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.4 * rating
        elif ratingName == 'Pss':
            value_returned = 0.9 * rating
        elif ratingName == 'Diq':
            value_returned = 0.95 * rating
    elif playerType == 'slasher':
        if ratingName == 'Dnk':
            value_returned = 1.5 * rating
        elif ratingName == 'Ins':
            value_returned = 1.4 * rating
        elif ratingName == 'Jmp':
            value_returned = 1.35 * rating
        elif ratingName == 'Spd':
            value_returned = 1.35 * rating
        elif ratingName == 'Fg':
            value_returned = 1.3 * rating
        elif ratingName == 'Tp':
            value_returned = 0.95 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.3 * rating
        elif ratingName == 'Drb':
            value_returned = 1.2 * rating
        elif ratingName == 'Pss':
            value_returned = 0.9 * rating
    elif playerType == 'derozan':  
        if ratingName == 'Ft':
            value_returned = 1.5 * rating
        elif ratingName == 'Fg':
            value_returned = 1.6 * rating
        elif ratingName == 'Tp':
            value_returned = 0.5 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.5 * rating
        elif ratingName == 'Drb':
            value_returned = 1.3 * rating
    elif playerType == 'versatile':
        if ratingName == 'Drb':
            value_returned = 1.2 * rating
        elif ratingName == 'Pss':
            value_returned = 1.1 * rating
        elif ratingName == 'Dnk':
            value_returned = 1.35 * rating
        elif ratingName == 'Jmp':
            value_returned = 1.35 * rating
        elif ratingName == 'Spd':
            value_returned = 1.35 * rating
        elif ratingName == 'Ft':
            value_returned = 1.3 * rating
        elif ratingName == 'Fg':
            value_returned = 1.3 * rating
        elif ratingName == 'Tp':
            value_returned = 1.2 * rating
        elif ratingName == 'Reb':
            value_returned = 1.3 * rating
        elif ratingName == 'Oiq':
            value_returned = 1.3 * rating
        elif ratingName == 'Diq':
            value_returned = 1.25 * rating
    return value_returned



In [12]:
def getOvr(value):
    if value >= 68.:
        return round(value+8.)
    elif value >= 50.:
        return round(value+ (4. + ((value - 50.) * (4. / 18.))))
    elif value >= 42.:
        return round(value+ (-5. + (value -42.) * (10. / 8.)))
    elif value >= 31.:
        return round(value+ (-5. - (42.-value) * (5. / 11.) ))
    else:
        return round(value -10.)
   

In [13]:
def generate_class():
    ratings_list = ['Stre','Spd','Jmp','Endu','Ins','Dnk','Ft','Fg','Tp','Oiq','Diq','Drb','Pss','Reb']

    athl_list = ['Stre','Spd','Jmp','Endu','Dnk']
    shoot_list = ['Ft','Fg','Tp']
    skill_list = ['Oiq','Drb','Pss']

    class_df['Hgt'] = class_df['Hgt'].astype(int)


    class_df['ratingsAdjustment'] = class_df.apply(getBaselineAdj, axis=1)

    class_df['factorAthleticism'] = class_df.apply(getFactor, axis=1)
    class_df['factorShooting'] = class_df.apply(getFactor, axis=1)
    class_df['factorSkill'] = class_df.apply(getFactor, axis=1)
    class_df['factorD'] = class_df.apply(getFactor, axis=1)
    class_df['factorReb'] = class_df.apply(getFactor, axis=1)


    class_df['playerType'] = class_df.apply(lambda x: getType(x['Hgt']), axis=1)

    class_df['Stre'] = 37 + class_df['ratingsAdjustment']
    class_df['Spd'] = 40 + class_df['ratingsAdjustment']
    class_df['Jmp'] = 40 + class_df['ratingsAdjustment']
    class_df['Endu'] = 17 + class_df['ratingsAdjustment']
    class_df['Ins'] = 27 + class_df['ratingsAdjustment']
    class_df['Dnk'] = 27 + class_df['ratingsAdjustment']
    class_df['Ft'] = 32 + class_df['ratingsAdjustment']
    class_df['Fg'] = 32 + class_df['ratingsAdjustment']
    class_df['Tp'] = 32 + class_df['ratingsAdjustment']
    class_df['Oiq'] = 22 + class_df['ratingsAdjustment']
    class_df['Diq'] = 22 + class_df['ratingsAdjustment']
    class_df['Drb'] = 37 + class_df['ratingsAdjustment']
    class_df['Pss'] = 37 + class_df['ratingsAdjustment']
    class_df['Reb'] = 37 + class_df['ratingsAdjustment']
    
    for rating in ratings_list:
        class_df[rating] = class_df.apply(lambda x: getFuzz(x[rating]), axis=1)

    for ath_rating in athl_list:
        class_df[ath_rating] = class_df[ath_rating] * class_df['factorAthleticism']

    for shoot_rating in shoot_list:
        class_df[shoot_rating] = class_df[shoot_rating] * class_df['factorShooting']

    for skill_rating in skill_list:
        class_df[skill_rating] = class_df[skill_rating] * class_df['factorSkill']

    class_df['Diq'] = class_df['Diq'] * class_df['factorD']
    class_df['Reb'] = class_df['Reb'] * class_df['factorReb']

    for rating in ratings_list:
        class_df[rating] = class_df.apply(lambda x: getTypeFactor(x[rating], x['playerType'], rating), axis=1)
    
    for rating in ratings_list:
        class_df[rating] = class_df.apply(lambda x: getPretty(x[rating]), axis=1)

    class_final = class_df[['Name','playerType','Hgt','Stre','Spd','Jmp','Endu','Ins','Dnk','Ft','Fg','Tp','Oiq','Diq','Drb','Pss','Reb']]
    class_final['Ovr_stp1'] = ((5 *class_final['Hgt'] + 1 *class_final['Stre'] + 4 *class_final['Spd'] + 2 *class_final['Jmp'] + 1 *class_final['Endu'] + 1 *class_final['Ins'] + 2 *class_final['Dnk'] + 1 *class_final['Ft'] + 1 *class_final['Fg'] + 3 *class_final['Tp'] + 7 *class_final['Oiq'] + 3 *class_final['Diq'] + 3 *class_final['Drb'] + 3 *class_final['Pss'] + 1 * class_final['Reb']) / 38.)
    
    class_final['Ovr'] = class_final.apply(lambda x: getOvr(x['Ovr_stp1']), axis=1)
    
    class_final['Hometown'] = ''
    class_final['HS'] = ''
        
    class_final = class_final.sort_values('Ovr', ascending=False).reset_index()

    class_final = class_final[['Name','playerType','Ovr', 'Ovr_stp1','Hgt','Stre','Spd','Jmp','Endu','Ins','Dnk','Ft','Fg','Tp','Oiq','Diq','Drb','Pss','Reb', 'Hometown', 'HS']]
    
    return class_final

In [14]:
class_average_top_5 = 0.
counter = 0
class_variance_top_5 = 9999.
while (True):
    counter+=1
    class_done = generate_class()
    class_average_top_5 =  class_done['Ovr'].head(5).mean(axis=0)
    class_variance_top_5 =  class_done['Ovr'].head(5).var(axis=0)
    
    five_star_end = class_done.loc[29,'Ovr']
    rec50 = class_done.loc[49,'Ovr']
    four_star_end = class_done.loc[149,'Ovr']
    
    three_star_end = class_done.loc[249,'Ovr']
    if(class_done['Hgt'].head(8).max() >= 62):
        if ((class_average_top_5 >= 68.0) & (class_average_top_5 <= 71.)):
            if class_variance_top_5 <=10.:
                if (five_star_end  <= 56.) & (five_star_end >= 52.):
                    if (four_star_end  <= 38.) & (four_star_end >= 34.):
                        if (three_star_end  <= 30.) & (three_star_end >= 26.):
                            if (rec50 <= 50.) & (rec50 >= 46):
                                print(counter)
                                break
                            else:
                                print("Recruit 50 not in bounds")
                        else:
                            print("3 star end not in bounds")
                    else:
                        print("4 star end not in bounds")
                else:
                    print("5 star end not in bounds")
            else:
                print("Top 5 variance > 10: " + str(class_variance_top_5))
        else:
            print("Top 5 average not in bounds: " + str(class_average_top_5))
    else:
        print("Max < 64: " + str(class_done['Hgt'].head(8).max()))
    #print(class_done.head(30))
        

C:\Users\arvin\Anaconda3\lib\site-packages\ipykernel_launcher.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\arvin\Anaconda3\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\arvin\Anaconda3\lib\site-packages\ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 64.2
Top 5 average not in bounds: 62.2
Max < 64: 60
Top 5 average not in bounds: 64.6
Max < 64: 57
Max < 64: 60
Top 5 average not in bounds: 64.8
Top 5 average not in bounds: 63.4
Top 5 average not in bounds: 62.8
Max < 64: 55
Max < 64: 52
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 67.8
Max < 64: 56
Top 5 average not in bounds: 61.8
Top 5 average not in bounds: 65.4
Top 5 average not in bounds: 66.0
Top 5 average not in bounds: 62.0
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 65.2
Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 63.4
Max < 64: 60
Top 5 average not in bounds: 63.0
Max < 64: 61
Max < 64: 59
Top 5 average not in bounds: 63.8
Top 5 average not in bounds: 63.4
Max < 64: 56
Max < 64: 55
Top 5 average not in bounds: 67.0
Max < 64: 60
Top 5 average not in bounds: 66.6
Top 5 average not in bounds: 65.4
Max < 64: 56
Top 5 average not in bounds: 66.2
Max < 64: 57
To

Top 5 average not in bounds: 64.4
Max < 64: 58
Top 5 average not in bounds: 63.8
Top 5 average not in bounds: 64.8
Top 5 average not in bounds: 67.6
Top 5 average not in bounds: 62.8
Top 5 average not in bounds: 65.4
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 63.4
Top 5 average not in bounds: 62.0
Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 67.2
Top 5 average not in bounds: 67.4
Top 5 average not in bounds: 65.2
Top 5 average not in bounds: 62.6
Top 5 average not in bounds: 66.0
Top 5 average not in bounds: 66.2
Top 5 average not in bounds: 63.4
Max < 64: 58
Top 5 average not in bounds: 61.4
Max < 64: 60
Top 5 average not in bounds: 65.6
Max < 64: 54
Top 5 average not in bounds: 67.4
Max < 64: 56
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 66.4
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 66.4
Top 5 average not in bounds: 63.8
Top 5 average not

Top 5 average not in bounds: 63.4
Recruit 50 not in bounds
Top 5 average not in bounds: 64.6
Top 5 average not in bounds: 62.4
Top 5 average not in bounds: 59.6
Top 5 average not in bounds: 66.4
Max < 64: 56
Max < 64: 58
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 62.4
Top 5 average not in bounds: 64.6
Max < 64: 58
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 64.4
Max < 64: 60
Max < 64: 57
Max < 64: 60
Top 5 average not in bounds: 64.8
Top 5 average not in bounds: 62.8
Top 5 average not in bounds: 64.2
Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 64.8
Top 5 average not in bounds: 67.0
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 62.8
Max < 64: 57
Top 5 average not in bounds: 67.0
Recruit 50 not in bounds
Max < 64: 55
Top 5 average not in bounds: 64.0
Max < 64: 61
Top 5 average not in bounds: 65.0
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 62.6
Top 5 average not

Top 5 average not in bounds: 61.6
Top 5 average not in bounds: 67.4
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 62.6
Top 5 average not in bounds: 64.4
Max < 64: 56
Max < 64: 61
Top 5 average not in bounds: 67.2
Top 5 average not in bounds: 66.0
Top 5 average not in bounds: 66.6
Top 5 average not in bounds: 65.6
Max < 64: 57
Top 5 average not in bounds: 67.2
Top 5 average not in bounds: 66.2
Top 5 average not in bounds: 65.0
Top 5 average not in bounds: 66.4
Top 5 average not in bounds: 65.0
Top 5 average not in bounds: 63.6
Max < 64: 52
Top 5 average not in bounds: 62.6
Top 5 variance > 10: 12.199999999999998
Top 5 average not in bounds: 66.2
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 63.8
Top 5 average not in bounds: 62.0
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 66.4
Max < 64: 57
Top 5 average not in bounds: 64.6
Top 5 average not in bounds: 63.2
Top 5 avera

Top 5 average not in bounds: 62.0
Max < 64: 56
Top 5 average not in bounds: 65.4
Top 5 average not in bounds: 63.4
Top 5 average not in bounds: 62.6
Max < 64: 58
Top 5 average not in bounds: 63.6
Max < 64: 57
Max < 64: 54
Top 5 average not in bounds: 62.0
Top 5 average not in bounds: 65.6
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 64.2
Top 5 average not in bounds: 66.0
Top 5 average not in bounds: 66.0
Top 5 average not in bounds: 62.8
Top 5 average not in bounds: 66.8
Top 5 average not in bounds: 63.4
Max < 64: 55
Top 5 average not in bounds: 64.8
Top 5 average not in bounds: 65.0
Top 5 average not in bounds: 63.8
Top 5 average not in bounds: 63.4
Top 5 average not in bounds: 65.2
Top 5 average not in bounds: 62.0
Max < 64: 58
Top 5 average not in bounds: 63.4
Top 5 average not in bounds: 67.2
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 65.6
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 64.4
Max 

Max < 64: 60
Top 5 average not in bounds: 63.4
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 64.2
Top 5 average not in bounds: 62.8
Top 5 average not in bounds: 62.6
Top 5 average not in bounds: 63.2
Max < 64: 56
Max < 64: 57
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 64.8
Top 5 average not in bounds: 63.8
Max < 64: 56
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 66.0
Max < 64: 61
Recruit 50 not in bounds
Top 5 average not in bounds: 64.6
Max < 64: 57
Top 5 average not in bounds: 66.2
Max < 64: 59
Top 5 average not in bounds: 66.2
Max < 64: 55
Top 5 average not in bounds: 61.8
Max < 64: 56
Top 5 average not in bounds: 66.6
Top 5 average not in bounds: 73.2
Max < 64: 56
Max < 64: 58
Max < 64: 51
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 65.8
Top 5 average not in bounds: 66.6
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 64.2
Max < 64: 58
Top 5 average not in bou

Max < 64: 59
Top 5 average not in bounds: 65.8
Top 5 average not in bounds: 63.8
Max < 64: 57
Top 5 average not in bounds: 64.6
Max < 64: 58
Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 67.4
Top 5 average not in bounds: 67.6
Top 5 average not in bounds: 61.2
Max < 64: 60
Top 5 average not in bounds: 61.8
Top 5 average not in bounds: 63.0
Max < 64: 58
Top 5 average not in bounds: 64.0
Top 5 average not in bounds: 65.8
Top 5 average not in bounds: 64.8
Max < 64: 59
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 61.6
Top 5 average not in bounds: 64.2
Max < 64: 56
Top 5 average not in bounds: 64.6
Top 5 average not in bounds: 65.0
Top 5 average not in bounds: 64.2
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 66.8
Max < 64: 56
Top 5 average not in bounds: 63.0
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 62.6
Top 5 average not in bounds: 65.4
Top 5 average not in bounds: 64.6
Top 5 average not in bounds: 66.0
Top 5 averag

Top 5 average not in bounds: 66.4
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 64.6
Recruit 50 not in bounds
Top 5 average not in bounds: 66.4
Top 5 average not in bounds: 64.2
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 63.6
Max < 64: 57
Max < 64: 60
Max < 64: 58
Top 5 average not in bounds: 60.8
Top 5 average not in bounds: 64.8
Max < 64: 53
Top 5 average not in bounds: 65.2
Top 5 average not in bounds: 62.0
Top 5 average not in bounds: 65.0
Top 5 average not in bounds: 63.6
Top 5 average not in bounds: 64.2
Top 5 average not in bounds: 64.4
Top 5 average not in bounds: 60.8
Top 5 average not in bounds: 61.0
Top 5 average not in bounds: 63.2
Top 5 average not in bounds: 62.8
Top 5 average not in bounds: 62.8
Top 5 average not in bounds: 65.6
Top 5 average not in bounds: 65.8
Top 5 average not in bounds: 65.2
Max < 64: 58
Max < 64: 56
Top 5 average not in bounds: 61.2
Top 5 variance > 10: 24.700000000000006
Max < 64: 58
Top 5 variance > 10: 26.0
To

In [15]:
class_done.head(30)

,Name,playerType,Ovr,Ovr_stp1,Hgt,Stre,Spd,Jmp,Endu,Ins,...,Ft,Fg,Tp,Oiq,Diq,Drb,Pss,Reb,Hometown,HS
0,Grant Wilson,captain,72,64.447368,35,71,90,73,56,58,...,59,67,63,74,39,73,82,64,,
1,Cal Smith,captain,69,62.184211,51,69,79,73,60,51,...,49,51,55,67,59,67,69,41,,
2,Darryl Battle,versatile,68,61.657895,50,61,83,91,36,38,...,52,64,53,53,50,85,62,59,,
3,Jesse Jones,captain,67,60.657895,51,65,79,84,61,54,...,62,61,65,58,52,58,53,53,,
4,Parker Battle,fundamentals,64,58.421053,69,52,44,44,25,57,...,88,62,56,52,47,83,82,57,,
5,Steve Kirby,captain,63,57.473684,67,60,67,62,40,38,...,58,58,56,54,61,52,56,56,,
6,Miroslav Sipahi,captain,61,55.736842,33,60,70,66,43,49,...,58,54,61,53,71,64,56,66,,
7,Max Provo,defense only,61,55.631579,46,74,94,92,33,56,...,27,28,29,46,63,51,60,79,,
8,Stephen Gak,fundamentals,60,54.921053,60,60,41,43,18,60,...,52,40,39,58,54,79,83,64,,
9,Vincent Weis,versatile,60,55.315789,38,58,71,71,32,43,...,37,43,41,57,57,69,60,89,,


In [16]:

class_done['Ovr'].head(5).mean(axis=0)

68.0

In [17]:
class_done['Ovr'].head(5).var(axis=0)

8.5

In [18]:
class_done.loc[29,'Ovr']

55

In [19]:
class_done.loc[149,'Ovr']

35

In [20]:
class_done.loc[249,'Ovr']

28

In [21]:
#pd.options.display.max_rows = 999

In [22]:
test = class_done.loc[(class_done['Name'] == "Marquis Levett") & (class_done['Hgt'] == 42)]
test

,Name,playerType,Ovr,Ovr_stp1,Hgt,Stre,Spd,Jmp,Endu,Ins,...,Ft,Fg,Tp,Oiq,Diq,Drb,Pss,Reb,Hometown,HS


In [23]:
def generateLoc(locations,i,name):
    keys = list(locations.keys())
    rand = random.random()
    for i in range(0,len(keys)):
        if keys[i] == rand:
            key = keys[i]
            break
        if keys[i] > rand:
            key = keys[i-1]
            break
    return(locations[key])

In [24]:
def haversine(coords1,coords2):
    lat1 = (coords1[0])*pi/180
    lat2 = (coords2[0])*pi/180
    long1 = (coords1[1])*pi/180
    long2 = (coords2[1])*pi/180
    
    latDiff = lat1-lat2
    longDiff = long1-long2
    
    hav = ((1-cos(latDiff))/2) + ((cos(lat1))*(cos(lat2))*((1-cos(longDiff))/2)) 
    return 7917.5*sqrt(asin(hav))

In [25]:
def generateHS(location,t50,t250,t1500Cities,t1500Coords,i,name, cityToCoords):
    rng = 1
    
    if i <= 250:
    
        if i <= 30:
            leaveState = random.randint(1,15)
            rng = 15
        elif i <= 90:
            leaveState = random.randint(1,20)
            rng = 20
        elif i <= 150:
            leaveState = random.randint(1,25)
            rng = 25
        else:
            leaveState = random.randint(1,30)
            rng = 30
        if type(location) == str:
            if i in internationals:
                if (i <= 150 and leaveState > 4) or (i > 150 and leaveState > 5): 
                    leaveState = 10
                else:
                    return
            else:
                city = location.split(", ")[-2]
                state = location.split(", ")[-1]
        else:
            city = location[0][0]
            state = location[0][1]
            coords = location[1]
              
        if leaveState == 10:
            random.shuffle(t50)
            random.shuffle(t50)
            random.shuffle(t50)
            return t50[0]
        
        if i <= 30:
            inStateT250 = random.randint(1,5)
            rng = 5
        elif i <= 75:
            inStateT250 = random.randint(1,7)
            rng = 7
        elif i <= 100:
            inStateT250 = random.randint(1,10)
            rng = 10
        elif i <= 150:
            inStateT250 = random.randint(1,15)
            rng = 15
        else:
            inStateT250 = random.randint(1,20)
            rng = 20
        
        if inStateT250 == 5:
            eligibleSchools = list(filter(lambda x: x[1] == state, t250))
            if len(eligibleSchools) != 0:
                random.shuffle(eligibleSchools)
                random.shuffle(eligibleSchools)
                random.shuffle(eligibleSchools)
                return eligibleSchools[0][0]
    else:
        if type(location) != str:
            city = location[0][0]
            state = location[0][1]
            coords = location[1]
        else:
            city = location.split(", ")[-2]
            state = location.split(", ")[-1]

    if city + ", " + state in t1500Cities:
        #print(city + ", " + state)
        #print("in CITY")
        eligibleSchools = t1500Cities[city + ", " + state]
        random.shuffle(eligibleSchools)
        random.shuffle(eligibleSchools)
        random.shuffle(eligibleSchools)
        return eligibleSchools[0]
    else:
        if city + ", " + state in cityToCoords:
            #print("city not Found")
            coords = cityToCoords[city + ", " + state][1]
            keys = list(t1500Coords.keys())
            keys.sort(key= lambda x: haversine(coords,(x[0],x[1])))
            eligibleSchools = t1500Coords[keys[0]]
            random.shuffle(eligibleSchools)
            random.shuffle(eligibleSchools)
            random.shuffle(eligibleSchools)
            return eligibleSchools[0]
         
    return "N/A"

In [26]:
loc = ("location_info/NCBCA Recruit Locations Generator.xlsx") 
wb = xlrd.open_workbook(loc) 
sheet = wb.sheet_by_index(1)

In [27]:
loc2 = ("location_info/High School Randomizer.xlsx")
wb2 = xlrd.open_workbook(loc2) 
topFifty = wb2.sheet_by_index(4)
topTwoFifty = wb2.sheet_by_index(5)
topFifteenHundred = wb2.sheet_by_index(6)

In [28]:
t50 = []
t250 = []
t1500Cities = {}
t1500Coords = {}

In [29]:
for i in range(1,51):
    t50.append(topFifty.cell_value(i,1))
    
t50 = 7 * t50[0:5] + 4 * t50[5:15] + 2 * t50[15:25] + t50[25:] #WEIGHTS

In [30]:
for i in range(0,449):
    if (topTwoFifty.cell_value(i,1), topTwoFifty.cell_value(i,2)) not in t250:
        t250.append((topTwoFifty.cell_value(i,1), topTwoFifty.cell_value(i,2)))
        
t250 = 5 * t250[0:25] + 3 * t250[25:50] + 2 * t250[50:100] + t250[100:] #WEIGHTS

In [31]:
for i in range(1,1501):
    city = topFifteenHundred.cell_value(i,4)
    state = topFifteenHundred.cell_value(i,2)
    HS = topFifteenHundred.cell_value(i,1)
    coords = (topFifteenHundred.cell_value(i,7),topFifteenHundred.cell_value(i,8))
    
    if city + ", " + state not in t1500Cities:
        t1500Cities[city + ", " + state] = [HS]
    else:
        t1500Cities[city + ", " + state].append(HS)
        
    if coords not in t1500Coords:
        t1500Coords[coords] = [HS]
    else:
        t1500Coords[coords].append(HS)

In [32]:
locations = {}
cityToCoords = {}
for i in range(1,19360):
    locations[sheet.cell_value(i,5)] = ((sheet.cell_value(i,1),sheet.cell_value(i,0)),((sheet.cell_value(i,6),(sheet.cell_value(i,7)))))
    cityToCoords[(sheet.cell_value(i,1) + ", " + sheet.cell_value(i,0))] = ((sheet.cell_value(i,1),sheet.cell_value(i,0)),((sheet.cell_value(i,6),(sheet.cell_value(i,7)))))

In [33]:
for index, row in class_done.iterrows():
    if index+1 not in internationals:
        location = generateLoc(locations, index+1, row['Name'])
        class_done.at[index, "Hometown"] = location[0][0] + ", " + location[0][1]
    else:
        class_done.at[index, "Hometown"] = "INTERNATIONAL, LOCATION"
        location = "INTERNATIONAL, LOCATION"
    #print(location)
    hs = generateHS(location, t50,t250,t1500Cities,t1500Coords,index+1,row['Name'], cityToCoords)
    #print(location, index+1, hs)
    if hs is None:
        hs = "EUROLEAGUE"
    class_done.at[index, "HS"] = hs
class_done["Pos"] = ""

In [34]:
class_done.head(30)

,Name,playerType,Ovr,Ovr_stp1,Hgt,Stre,Spd,Jmp,Endu,Ins,...,Fg,Tp,Oiq,Diq,Drb,Pss,Reb,Hometown,HS,Pos
0,Grant Wilson,captain,72,64.447368,35,71,90,73,56,58,...,67,63,74,39,73,82,64,"Norfolk, VA",Paul VI (Fairfax),
1,Cal Smith,captain,69,62.184211,51,69,79,73,60,51,...,51,55,67,59,67,69,41,"Corpus Christi, TX",Los Fresnos,
2,Darryl Battle,versatile,68,61.657895,50,61,83,91,36,38,...,64,53,53,50,85,62,59,"Hampton, VA",Menchville (Newport News),
3,Jesse Jones,captain,67,60.657895,51,65,79,84,61,54,...,61,65,58,52,58,53,53,"Lexington, KY",Lexington Catholic (Lexington),
4,Parker Battle,fundamentals,64,58.421053,69,52,44,44,25,57,...,62,56,52,47,83,82,57,"INTERNATIONAL, LOCATION",Sierra Canyon (Chatsworth),
5,Steve Kirby,captain,63,57.473684,67,60,67,62,40,38,...,58,56,54,61,52,56,56,"Rutherfordton, NC",Shelby,
6,Miroslav Sipahi,captain,61,55.736842,33,60,70,66,43,49,...,54,61,53,71,64,56,66,"Slidell, LA",Carver Collegiate Academy (New Orleans),
7,Max Provo,defense only,61,55.631579,46,74,94,92,33,56,...,28,29,46,63,51,60,79,"Buffalo, NY",Canisius (Buffalo),
8,Stephen Gak,fundamentals,60,54.921053,60,60,41,43,18,60,...,40,39,58,54,79,83,64,"Mendota, CA",San Joaquin Memorial (Fresno),
9,Vincent Weis,versatile,60,55.315789,38,58,71,71,32,43,...,43,41,57,57,69,60,89,"Pawtucket, RI",Bishop Hendricken (Warwick),


In [35]:
def update(class_done, json):
    for player in json.get("players"):
        ratings = player["ratings"][-1];
        if ratings["season"] == year - diff and player["tid"] == tid and year - player.get("born")["year"] == age:
            name = player["firstName"] + " " + player["lastName"]
            new_rtgs = class_done.loc[(class_done['Name'] == name) & (class_done['Hgt'] == ratings["hgt"])]
            if(new_rtgs.empty):
                print(name + " could not be updated! Fix manually")
            else:
                class_done.loc[(class_done['Name'] == name) & (class_done['Hgt'] == ratings["hgt"]), "pos"] = calcPos(ratings)
                ratings["stre"] = int(new_rtgs["Stre"].iloc[0])
                ratings["spd"] = int(new_rtgs["Spd"].iloc[0])
                ratings["jmp"] = int(new_rtgs["Jmp"].iloc[0])
                ratings["endu"] = int(new_rtgs["Endu"].iloc[0])
                ratings["ins"] = int(new_rtgs["Ins"].iloc[0])
                ratings["dnk"] = int(new_rtgs["Dnk"].iloc[0])
                ratings["ft"] = int(new_rtgs["Ft"].iloc[0])
                ratings["fg"] = int(new_rtgs["Fg"].iloc[0])
                ratings["tp"] = int(new_rtgs["Tp"].iloc[0])
                ratings["oiq"] = int(new_rtgs["Oiq"].iloc[0])
                ratings["diq"] = int(new_rtgs["Diq"].iloc[0])
                ratings["drb"] = int(new_rtgs["Drb"].iloc[0])
                ratings["pss"] = int(new_rtgs["Pss"].iloc[0])
                ratings["reb"] = int(new_rtgs["Reb"].iloc[0])
                ratings["ovr"] = int(new_rtgs["Ovr"].iloc[0])
                player["city"] = str(new_rtgs["Hometown"].iloc[0].split(", ")[0])
                player["state"] = str(new_rtgs["Hometown"].iloc[0].split(", ")[1])
                player["college"] = new_rtgs["HS"].iloc[0]
                ratings["skills"] = []
                for badge in ['3', 'A', 'B', 'Di', 'Dp', 'Po', 'Ps', 'R']:
                    info = getAtts(badge)
                    atts = info[0]
                    weights = info[1]
                    cutoff = info[2]
                    rtgs = []
                    for att in atts:
                        rtgs.append(int(new_rtgs[att].iloc[0]))
                    if getComposite(rtgs, weights) > cutoff:
                        ratings["skills"].append(badge)

In [36]:
#update(class_done, obj)

In [37]:
#class_done.to_csv(str(year+years_in_future)+".csv")

In [38]:
#writeJSON(fileName, obj)

In [39]:
print(generateHS("Oakland, OK", t50,t250,t1500Cities,t1500Coords,1,"boi", cityToCoords))

Prosper
